In [1]:
%pip install semantic-kernel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade semantic-kernel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys

notebook_dir = os.path.abspath("C:\\Users\\katari.asish\\.vscode\\Semantic_Kernel\\Sk.ipynb")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

In [2]:
#import semantic_kernel as sk
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.prompt_template import PromptTemplateConfig
from dotenv import dotenv_values


config = dotenv_values("Configuration.env")
kernel = Kernel()


In [3]:
from semantic_kernel.contents.chat_history import ChatHistory  
from semantic_kernel.functions.kernel_arguments import KernelArguments  
from semantic_kernel.prompt_template.input_variable import InputVariable

In [4]:
ask = """
I want to get a list of ideas to become a cricket analyst.
"""

In [5]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel
import traceback

In [6]:
kernel.add_service(
    AzureChatCompletion(
        service_id="Cricket",
        deployment_name=config["AZURE_OPENAI_TEXT_DEPLOYMENT_NAME"],
        api_key=config["AZURE_OPENAI_API_KEY"],
        api_version=config["AZURE_OPENAI_API_VERSION"],
        endpoint=config["AZURE_OPENAI_ENDPOINT"]
    )
)

In [7]:
plugins_directory = "Prompt_template/"
summarize_plugin = kernel.add_plugin(plugin_name="SummarizePlugin", parent_directory=plugins_directory)
writer_plugin = kernel.add_plugin(
    plugin_name="WriterPlugin",
    parent_directory=plugins_directory,
)
text_plugin = kernel.add_plugin(plugin=TextPlugin(), plugin_name="TextPlugin")
cricket_func = KernelFunctionFromPrompt(
    function_name="Cricket",
    plugin_name="WriterPlugin",
    prompt="""
{{$input}}

Give ideas to become a cricket analyst.

""",
     prompt_execution_settings=AzureChatPromptExecutionSettings(
        service_id="Cricket",
        max_tokens=2000,
        temperature=0.8,
    ),
    description="Give ideas to become a cricket analyst.",
)
kernel.add_function(plugin_name="WriterPlugin", function=cricket_func)

for plugin_name, plugin in kernel.plugins.items():
   for function_name, function in plugin.functions.items():
       print(f"Plugin: {plugin_name}, Function: {function_name}")



Plugin: SummarizePlugin, Function: Summarize
Plugin: WriterPlugin, Function: Brainstorm
Plugin: WriterPlugin, Function: Cricket
Plugin: TextPlugin, Function: lowercase
Plugin: TextPlugin, Function: trim
Plugin: TextPlugin, Function: trim_end
Plugin: TextPlugin, Function: trim_start
Plugin: TextPlugin, Function: uppercase


In [8]:
from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel, service_id="Cricket")

In [9]:
sequential_plan = await planner.create_plan(goal=ask)

In [10]:
print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

The plan's steps are:
- Convert a string to lowercase using TextPlugin-lowercase with parameters: {'input': '$INPUT'}
- Trim whitespace from the start and end of a string using TextPlugin-trim with parameters: {'input': '$LOWER_GOAL'}
- Give ideas to become a cricket analyst using WriterPlugin-Cricket with parameters: {'input': '$TRIMMED_GOAL'}


In [11]:
result = await sequential_plan.invoke(kernel)

print(result)

1. Gain a deep understanding of cricket: To become a cricket analyst, you need to have an in-depth knowledge of the game, its rules, technicalities, and strategies. You need to watch as many matches as you can, read books, blogs, and articles related to cricket, and follow the latest news and updates.

2. Develop analytical skills: As a cricket analyst, you need to have excellent analytical skills to examine and interpret the data and statistics related to the game. You should be able to identify patterns, trends, and anomalies in the game, and provide insights and recommendations accordingly.

3. Good Communication: Communication is another important skill that you will need as a cricket analyst. You must be able to express your thoughts and ideas clearly and succinctly, both orally and in writing.

4. Technology: Being familiar with the latest technologies, software, and tools used in cricket analysis is essential. You should have a good understanding of statistical software packages